In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/TinArmEngineering/tinarm.git (to revision 0.1.49) to /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-vofw6d_h
  Running command git clone --filter=blob:none --quiet https://github.com/TinArmEngineering/tinarm.git /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-vofw6d_h
  Running command git checkout -q a2fdc7ce19efad177874d9b06fdab31b85f00681
  Resolved https://github.com/TinArmEngineering/tinarm.git to commit a2fdc7ce19efad177874d9b06fdab31b85f00681
  Preparing metadata (setup.py) ... done


In [2]:
import matplotlib.pyplot as plt
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity 
from tinarm.api import JOB_STATUS

import tinarm
import logging
import time
import requests
import uuid
import random
import yaml
import pandas as pd
import pint_pandas
import numpy as np

In [3]:

LOGGING_LEVEL = logging.INFO
STATUS_JOB = {value:key for key,value in JOB_STATUS.items()}

In [4]:

### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

2024-01-25 11:34:01,098 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - tinarm version 0.1
2024-01-25 11:34:01,099 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - pint_pandas version 0.5


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [5]:
with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [6]:
stator_parameters = {
    "slot_liner_thikness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [7]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [8]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [9]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [10]:


current_density_grid, speed_grid = np.meshgrid(np.linspace(0, 10, 5) * q.A/q.mm**2, np.linspace(10, 3000, 5) * q.rpm)
current_density_grid.reshape(-1), speed_grid.reshape(-1)

(array([ 0. ,  2.5,  5. ,  7.5, 10. ,  0. ,  2.5,  5. ,  7.5, 10. ,  0. ,
         2.5,  5. ,  7.5, 10. ,  0. ,  2.5,  5. ,  7.5, 10. ,  0. ,  2.5,
         5. ,  7.5, 10. ]) <Unit('ampere / millimeter ** 2')>,
 array([  10. ,   10. ,   10. ,   10. ,   10. ,  757.5,  757.5,  757.5,
         757.5,  757.5, 1505. , 1505. , 1505. , 1505. , 1505. , 2252.5,
        2252.5, 2252.5, 2252.5, 2252.5, 3000. , 3000. , 3000. , 3000. ,
        3000. ]) <Unit('revolutions_per_minute')>)

In [11]:
operating_points = [{
    'current_density': cd,
    'current_angle': 255 * q.degrees,
    "simulated_speed": sp
    } for cd, sp in zip(current_density_grid.reshape(-1), speed_grid.reshape(-1))]



In [12]:
class Machine(object):
    def __init__(self, stator, rotor, winding):

        self.stator = stator
        self.rotor = rotor
        self.winding = winding

    def __repr__(self) -> str:
        return f"Machine({self.stator}, {self.rotor}, {self.winding})"
    
    def to_api(self):
        stator_api = [NameQuantityPair("stator",
                                        k,
                                        Quantity(*self.stator[k].to_tuple())) for k in  self.stator]
        rotor_api = [NameQuantityPair("rotor",
                                        k,
                                        Quantity(*self.rotor[k].to_tuple())) for k in  self.rotor]
        winding_api = [NameQuantityPair("winding",
                                        k,
                                        Quantity(*self.winding[k].to_tuple())) for k in  self.winding]
        data = []
        data.extend(list(x.to_dict() for x in stator_api))
        data.extend(list(x.to_dict() for x in rotor_api))
        data.extend(list(x.to_dict() for x in winding_api))
        return data
        
    
class Job(object):
    def __init__(self, machine: Machine, operating_point, simulation, title=None):
        if title is None:
            self.title = self.generate_title()
        else:
            self.title = title
        self.type = "electromagnetic_spmbrl_fscwseg"
        self.status = 0
        self.machine = machine
        self.operating_point = operating_point
        self.simulation = simulation
        
    def __repr__(self) -> str:
        return f"Job({self.machine}, {self.operating_point}, {self.simulation})"
    
    def generate_title(self):
        "gets a random title from the wordlists"
        random_offset = random.randint(500, 286797)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Adjectives-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        adjective = random.choice(response.text.splitlines()[1:-1])
        random_offset = random.randint(500, 871742)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Nouns-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        noun = random.choice(response.text.splitlines()[1:-1])
        title = f"{adjective}-{noun}"
        return title 

    def to_api(self):
        job =  {"status": 0,
                "title": self.title,
                "type": self.type,
                "tasks": 11,
                "data": []}
        
        operating_point_api = [NameQuantityPair("operating_point",
                                        k,
                                        Quantity(*self.operating_point[k].to_tuple())) for k in  self.operating_point]
        
        simulation_api = [NameQuantityPair("simulation",
                                        k,
                                        Quantity(*self.simulation[k].to_tuple())) for k in  self.simulation]
        job["data"].extend(list(x.to_dict() for x in operating_point_api))
        job["data"].extend(list(x.to_dict() for x in simulation_api))
        job["data"].extend(self.machine.to_api())
        return job
    
    def run(self):
        pass

In [13]:
class Api:
    """
    The TAE User API
    """

    def __init__(self, root_url, api_key, org_id=None):
        """
        Initialize the API
        """
        self._root_url = root_url
        self._api_key = api_key
        self._org_id = org_id
        self._node_id = None

        logger.info(f"root_url: {self._root_url}")

    def get_job(self, job_id):
        """
        Get a job from the TAE API
        """
        response = requests.get(
            url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
        )
        response.raise_for_status()
        return response.json()
    
    def create_job(self, job):
        """
        Create a job for the TAE API
        """
        response = requests.post(
            url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
            json=job.to_api()
        )
        response.raise_for_status()
        if response.status_code == 200:
            job.id = response.json()["id"]
        return response.json()
    
    def update_job_status(self, job_id, status):
        """
        Update a job status
        """
        url = f"{self._root_url}/jobs/{job_id}/status/{status}?node_id={self._node_id}&apikey={self._api_key}"
        logger.info(f"Updating job status: {url}")

        response = requests.put(url=url)
        return response.json()
    
    

In [14]:
api = Api(config["root_url"], config["api_key"], config["org_id"])

2024-01-25 11:34:10,025 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [15]:
print('amazing ({current_density:2.4~P}, {simulated_speed:2.4~P})'.format(**operating_points[0]))

amazing (0.0 A/mm², 10.0 rpm)


In [16]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)



jobs = [Job(m1, op, simulation_parameters, title='amazing ({current_density:2.4~P}, {simulated_speed:2.4~P})'.format(**op)) for op in operating_points]



jobs_mapping = map(lambda x: api.create_job(x), jobs)
jobs_result = list(jobs_mapping)

In [17]:
status = [STATUS_JOB[api.get_job(j['id'])['status']] for j in jobs_result]
print(status)


In [19]:
for j in jobs_result:
    res = api.update_job_status(j['id'], JOB_STATUS['QueuedForMeshing'])
    time.sleep(0.5)
    print(res)

2024-01-25 11:34:58,834 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65b2392a481c1394ee4e82d9/status/10?node_id=None&apikey=668952058c490d0a296da0abb9660534
{'id': '65b2392a481c1394ee4e82d9', 'owner_org_id': '65a9291658b95253163001ab', 'owner_user_id': '659bf5a7b7135efed48311bd', 'creation_date': '2024-01-25T10:34:18Z', 'status': 10, 'title': 'amazing (0.0 A/mm², 10.0 rpm)', 'type': 'electromagnetic_spmbrl_fscwseg', 'node_id': 'None', 'data': [{'section': 'operating_point', 'name': 'current_density', 'value': {'magnitude': 0, 'units': [{'name': 'ampere', 'exponent': 1}, {'name': 'millimeter', 'exponent': -2}]}}, {'section': 'operating_point', 'name': 'current_angle', 'value': {'magnitude': 255, 'units': [{'name': 'degree', 'exponent': 1}]}}, {'section': 'operating_point', 'name': 'simulated_speed', 'value': {'magnitude': 10, 'units': [{'name': 'revolutions_per_minute', 'exponent': 1

In [ ]:
j1_result = [api.update_job_status(j['id'], JOB_STATUS['QueuedForMeshing'])


while STATUS_JOB[api.get_job(j1.id)['status']] != 'Complete' or STATUS_JOB[api.get_job(j2.id)['status']] != 'Complete':
    print("job 1 {0} \t job 2 {1}".format(STATUS_JOB[api.get_job(j1.id)['status']], STATUS_JOB[api.get_job(j2.id)['status']] ))
    time.sleep(10)
time.sleep(1)

In [ ]:
j1_result = api.get_job(j1.id)
for artifact in j1_result['artifacts']:
    print(artifact['type'], artifact['url'], artifact['id'])

In [ ]:
j1_result_data = [art for art in j1_result['artifacts'] if art['type']=='RESULT_DATA'][0]
j1_df = pd.read_csv(j1_result_data['url'], header=[0, 1], index_col=[0, 1]).pint.quantify(level=-1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(j1_df['angles'], j1_df['Back_EMF_0'], label='Back_EMF_0')
ax.plot(j1_df['angles'], j1_df['Back_EMF_1'], label='Back_EMF_1')
ax.plot(j1_df['angles'], j1_df['Back_EMF_2'], label='Back_EMF_2')

In [ ]:
j2_result = api.get_job(j2.id)
for artifact in j2_result['artifacts']:
    print(artifact['type'], artifact['url'], artifact['id'])
time.sleep(1)    

In [ ]:
j2_result_data = [art for art in j2_result['artifacts'] if art['type']=='RESULT_DATA'][0]
j2_df = pd.read_csv(j2_result_data['url'], header=[0, 1], index_col=[0, 1]).pint.quantify(level=-1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Back_EMF_0'], label='Back_EMF_0', linestyle='--')
l2, = ax.plot(j1_df['angles'], j1_df['Back_EMF_1'], label='Back_EMF_1', linestyle='--',)
l3, = ax.plot(j1_df['angles'], j1_df['Back_EMF_2'], label='Back_EMF_2', linestyle='--')
ax.plot(j2_df['angles'], j2_df['Back_EMF_0'], label='nominal Back_EMF_0', color=l1.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_1'], label='nominal Back_EMF_1', color=l2.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_2'], label='nominal Back_EMF_2', color=l3.get_color())
ax.legend()

In [ ]:

fig = plt.figure(2)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Scaled Torque'], label='open cct Scaled Torque')
l2, = ax.plot(j2_df['angles'], j2_df['Scaled Torque'], label='nominal Scaled Torque')
mean_torque = j2_df['Scaled Torque'].mean()
ax.axhline( mean_torque, label=f'Scaled Torque {mean_torque:2.4~P}', linestyle='--')
target_torque = 24.26 * q.N * q.m
ax.axhline( target_torque, label=f'Target {target_torque:2.4~P}', color='k', linestyle='--')

ax.legend()

In [ ]:
m1_df = pd.read_csv("Machine_Back_emfs_Machine_1.csv", skiprows=1, header=[0, 1], index_col=[0]).pint.quantify(level=-1)
m1_df.head()


In [ ]:
m1_df['Angles electrical'] = pint_pandas.PintArray(m1_df.index * m1.rotor['number_poles'] / 2, dtype="pint[degree]")
m1_df.dtypes

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_0'], label='Back_EMF_0', linestyle='--')
l2, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_1'], label='Back_EMF_1', linestyle='--',)
l3, = ax.plot(j1_df['angles'], -j1_df['Back_EMF_2'], label='Back_EMF_2', linestyle='--')
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase A'], label='Phase A', color=l1.get_color())
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase B'], label='Phase B', color=l2.get_color())
ax.plot(m1_df['Angles electrical'].pint.to("radians"), m1_df['Phase C'], label='Phase C', color=l3.get_color())


In [ ]:
df_m1_phvol = pd.read_csv("m1_phase_voltage_nominal.csv",  header=[0, 1])
df_m1_phvol.head()

In [ ]:
from numpy import pi
fig = plt.figure()
ax = fig.add_subplot(111)
l1, =ax.plot(df_m1_phvol["Va"]["X"]*pi*5/180, -df_m1_phvol["Va"]["Y"],linestyle='--')
l2, =ax.plot(df_m1_phvol["Vb"]["X"]*pi*5/180, -df_m1_phvol["Vb"]["Y"],linestyle='--')
l3, =ax.plot(df_m1_phvol["Vc"]["X"]*pi*5/180, -df_m1_phvol["Vc"]["Y"],linestyle='--')
ax.plot(j2_df['angles'], j2_df['Back_EMF_0'], label='nominal Back_EMF_0', color=l1.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_1'], label='nominal Back_EMF_1', color=l2.get_color())
ax.plot(j2_df['angles'], j2_df['Back_EMF_2'], label='nominal Back_EMF_2', color=l3.get_color())
ax.legend()

In [ ]:
df_m1_phvol["Va"]["X"]*(pi*5/180)

In [ ]:
df_m1_torque = pd.read_csv("M1AverageTorqueNominal.csv", names=["Angles", "Torque"], header=0)
df_m1_torque.head()

In [ ]:
fig = plt.figure(2)
ax = fig.add_subplot(111)
l1, = ax.plot(j1_df['angles'], j1_df['Scaled Torque'], label='open cct Scaled Torque')
l2, = ax.plot(j2_df['angles'], j2_df['Scaled Torque'], label='nominal Scaled Torque')
mean_torque = j2_df['Scaled Torque'].mean()
ax.axhline( mean_torque, label=f'Scaled Torque {mean_torque:2.4~P}', linestyle='--')
target_torque = 24.26 * q.N * q.m
ax.axhline( target_torque, label=f'Target {target_torque:2.4~P}', color='k', linestyle='--')
ax.plot(df_m1_torque["Angles"]*pi*5/180, df_m1_torque["Torque"], label='nominal Torque')
ax.legend()